### author by yangshichen
### 注意：脚本仅供参考，使用前请仔细阅读

In [1]:
library(ggplot2)
library(future)
library(tidyverse)
library(ggpubr)
library(ggchicklet)
library(ggsci)
library(magrittr)
library(ggh4x)
library(rstatix)
library(ggsignif)
library(ggnewscale)
library(patchwork)
library(gapminder)
library(ggprism)
library(dplyr)
library(ggplotify)
library(readr)
library(arrow)
library(ggbreak)
library(lme4)
library(Matrix)
library(lmtest)
library(pbmcapply)
library(data.table)
library(arrow)
library(Seurat)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

载入程序包：‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract



载入程序包：‘rstatix’


The following object is masked from ‘package:stats’:

    filter



载入程序包：‘arrow’


The following object is masked from ‘package:magrittr’:

    is_in


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp


ggbreak v0.1.5 Learn more 

In [2]:
mytheme <- theme_prism(base_family="",base_fontface="plain") +
  theme(strip.text = element_text(size = 8,angle=10,vjust = 0.5,hjust = 0.5),
        axis.line = element_line(color = "black",size = 0.2),
        axis.ticks = element_line(size = 0.2),
        axis.text.y = element_text(color = "black",size = 6),
        axis.text.x = element_text(color = "black",size = 6, angle = 30,hjust = 1,vjust = 1),
        axis.title = element_text(color = "black",size = 10),
        legend.position = "none")

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


### scRNA-seq Data Process

#### h5ad转rds

In [1]:
import os
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import seaborn as sb
import seaborn as sns
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt
from scipy.io import mmread
from scipy.sparse import csr_matrix
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from scipy.sparse import issparse
import decoupler as dc
from sklearn.preprocessing import QuantileTransformer
from multiprocessing import Pool, cpu_count
from functools import partial
from sklearn.preprocessing import QuantileTransformer, StandardScaler
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
scaler = StandardScaler()

import warnings
warnings.filterwarnings("ignore")

In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=100, frameon=False)
sc._settings.ScanpyConfig.n_jobs=70

scanpy==1.9.8 anndata==0.9.2 umap==0.5.7 numpy==1.24.4 scipy==1.10.1 pandas==2.0.3 scikit-learn==1.3.2 statsmodels==0.14.1 pynndescent==0.5.13


In [4]:
adata = sc.read_h5ad("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/HIV-pbmc/pbmc_celltype.h5ad")
adata.X = adata.layers['counts'].copy()
adata

AnnData object with n_obs × n_vars = 2744009 × 21679
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'doublet', 'batch', 'sample', 'stage', 'experiments', 'age', 'total_counts_rp', 'pct_counts_rp', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ncRNA', 'pct_counts_ncRNA', 'total_counts_LOC', 'pct_counts_LOC', 'total_counts_erccs', 'pct_counts_erccs', 'celltype_L1', 'phase_ordered', 'celltype_L3', 'celltype_L1_5', 'celltype_L2'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mean', 'std'
    uns: 'age_colors', 'celltype_L2_colors', 'experiments_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'phase_ordered_colors', 'stage_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [5]:
#python
for ct in adata.obs["celltype_L3"].unique():
    print(f"Processing {ct}...")
    
    # 按 celltype 拆分
    adata_ct = adata[adata.obs["celltype_L3"] == ct].copy()
    
    # 提取 counts
    X = adata_ct.layers["counts"]   # 每个 celltype 子集的 raw counts
    genes = adata_ct.var_names
    
    # counts 矩阵 (行=基因，列=细胞)
    df = pd.DataFrame(
        X.T.toarray() if hasattr(X, "toarray") else X.T,
        index=genes,
        columns=adata_ct.obs_names)

    # metadata（行=细胞，列=信息）
    df_meta = adata_ct.obs.copy()
    df_meta.index.name = "cell_barcode"
    
    # 保存为 RDS 文件
    df.to_parquet(f"/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq/{ct}_count.parquet")
    df_meta.to_parquet(f"/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq/{ct}_meta.parquet")

Processing Switched_Memory_B-CD27...
Processing Unswitched_Memory_B-CD1C...
Processing Switched_Memory_B-CD86...
Processing Naive_B-TCL1A...
Processing Switched_Memory_B-IGHE...
Processing Aptypical_Memory_B-ITGAX...
Processing Transitional_B-NEIL1...
Processing Naive_B-IFIT3...
Processing Naive_B-IL6...
Processing Unswitched_Memory_B-IFIT3...
Processing Unswitched_Memory_B-JAM3...
Processing Plasma_B-IGHA1...
Processing Plasma_B-IGHG1...
Processing Plasmablast-MKI67...
Processing cMono-CD14...
Processing ncMono-FCGR3A...
Processing ncMono-C1QA...
Processing cDC2-CD1C...
Processing cMono-IFI44L...
Processing ncMono-IFIT1...
Processing pDC-AXL...
Processing ncMono-IFI44L...
Processing pDC-LILRA4...
Processing cDC1-BATF3...
Processing cMono-IL1B...
Processing CD4_Treg-FOXP3...
Processing CD4_Naive_T-CCR7...
Processing CD4_Tcm-CXCR5...
Processing CD4_Th22-CCR10...
Processing CD4_Th1-GZMK...
Processing CD4_Tcm-GPR183...
Processing CD4_Tem-CCR7neg...
Processing CD4_Treg-FCRL3...
Processing 

In [14]:
#R
save_dir <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq/"
count_files <- list.files(save_dir, pattern="_count\\.parquet$", full.names = TRUE)
celltypes <- sub("_count\\.parquet$", "", basename(count_files))

for(ct in celltypes){
    # 读取 counts 和 metadata
    counts <- as.data.frame(read_parquet(file.path(save_dir, paste0(ct, "_count.parquet"))))
    meta <- as.data.frame(read_parquet(file.path(save_dir, paste0(ct, "_meta.parquet"))))
    rownames(counts) <- counts[, 1]
    counts <- counts[,-1]
    rownames(meta) <- meta[, 1]
    meta <- meta[,-1]
  
    # 转成矩阵（SingleCellExperiment 要求 counts 是矩阵）
    counts_mat <- as.matrix(counts)
  
    # 创建 Seurat 对象
    sce <- CreateSeuratObject(counts = counts_mat,meta.data = meta)
    
    # 保存 RDS
    saveRDS(sce, file.path(save_dir, paste0(ct, ".rds")))
    paste0("Processing ",ct,"Done...")
    file.remove(file.path(save_dir, paste0(ct, "_count.parquet")))
    file.remove(file.path(save_dir, paste0(ct, "_meta.parquet")))
}

#### Clininfo

In [3]:
geneticsPCA <- as.data.frame(read.csv("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/input/genetics/genetics_PCA.csv"))
geneticsPCA <- geneticsPCA[,c(2,3,4)]
colnames(geneticsPCA) <- c("IID","geno_PC1","geno_PC2")
geneticsPCA

IID,geno_PC1,geno_PC2
<chr>,<dbl>,<dbl>
HD-H134,-0.01810910,0.07211560
HD-H135,-0.05604730,0.04604970
HD-H136,-0.03760770,0.02194770
HD-H137,-0.02201150,0.06467250
HD-H138,0.02504810,0.00261450
HD-H140,-0.05619350,0.12623300
HD-H141,-0.00489446,-0.13431300
HD-H143,0.04064280,-0.05805660
HD-H145,0.01114700,-0.01324910


In [4]:
info <- as.data.frame(read.csv("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/sample_information.csv",check.names = F,header = T))
colnames(info)[1] <- 'orig.ident'
info

orig.ident,gender,age,treat_time,stage,INSTI
<chr>,<int>,<int>,<int>,<chr>,<chr>
HD-H134,1,53,0,HDs,Null
HD-H135,1,26,0,HDs,Null
HD-H136,1,28,0,HDs,Null
HD-H137,1,31,0,HDs,Null
HD-H138,1,32,0,HDs,Null
HD-H140,1,28,0,HDs,Null
HD-H141,1,29,0,HDs,Null
HD-H143,1,33,0,HDs,Null
HD-H145,1,39,0,HDs,Null


In [5]:
clininfo <- cbind(info,geneticsPCA)
clininfo <- clininfo[,-7]
clininfo

orig.ident,gender,age,treat_time,stage,INSTI,geno_PC1,geno_PC2
<chr>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
HD-H134,1,53,0,HDs,Null,-0.01810910,0.07211560
HD-H135,1,26,0,HDs,Null,-0.05604730,0.04604970
HD-H136,1,28,0,HDs,Null,-0.03760770,0.02194770
HD-H137,1,31,0,HDs,Null,-0.02201150,0.06467250
HD-H138,1,32,0,HDs,Null,0.02504810,0.00261450
HD-H140,1,28,0,HDs,Null,-0.05619350,0.12623300
HD-H141,1,29,0,HDs,Null,-0.00489446,-0.13431300
HD-H143,1,33,0,HDs,Null,0.04064280,-0.05805660
HD-H145,1,39,0,HDs,Null,0.01114700,-0.01324910


#### Ge matrix & Covariates matrix

In [6]:
data.dir <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq/"
rds.files <- list.files(data.dir, pattern = ".rds$", full.names = TRUE)
save.dir <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/input/phenofile/"
common.genes.merged <- data.frame()

In [7]:
rds.files

[1] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//Aptypical_Memory_B-ITGAX.rds" 
 [2] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Naive_T-CCR7.rds"         
 [3] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Naive_T-SOX4.rds"         
 [4] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Tcm-CXCR5.rds"            
 [5] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Tcm-GPR183.rds"           
 [6] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Tcm-IFIT3.rds"            
 [7] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Tcm-SOX4.rds"             
 [8] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Tem-CCR7neg.rds"          
 [9] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Tfh_like-CXCR5.rds"       
[10] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Th-TNFRSF11A.rds"         
[11] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Th1-GZMK.rds"             
[12] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Th17-RORC.rds"            
[13] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Th22-CCR10.rds"           
[14] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Treg-FCRL3.rds"           
[15] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD4_Treg-FOXP3.rds"           
[16] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD8_CTL-GZMB.rds"             
[17] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD8_CTL-GZMK.rds"             
[18] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD8_Naive_T-CCR7.rds"         
[19] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD8_Tcm-GPR183.rds"           
[20] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD8_Tcm-GZMK.rds"             
[21] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD8_Tcm-IFI44L.rds"           
[22] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//CD8_Tem-GZMK.rds"             
[23] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//cDC1-BATF3.rds"               
[24] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//cDC2-CD1C.rds"                
[25] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//cMono-CD14.rds"               
[26] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//cMono-IFI44L.rds"             
[27] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//cMono-IL1B.rds"               
[28] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//Cycling_T-MKI67.rds"          
[29] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//gdT2-GZMH.rds"                
[30] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//gdT2-GZMK.rds"                
[31] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//gdT2-IL12RB2.rds"             
[32] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//Inflamed_NK_dim-IFI44L.rds"   
[33] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//MAIT-SLC4A10.rds"             
[34] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//Mature_NK_dim-FCGR3A.rds"     
[35] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//Naive_B-IFIT3.rds"            
[36] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//Naive_B-IL6.rds"              
[37] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//Naive_B-TCL1A.rds"            
[38] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//ncMono-C1QA.rds"              
[39] "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/scQTL/Data/scRNA-seq//ncMono-FCGR3A.rds"            
[40] "/media/AnalysisDi

In [8]:
for(i in rds.files){
    ser.tmp <- readRDS(i)
    metadata.tmp <- ser.tmp@meta.data[,c("sample", 'total_counts', "pct_counts_mt")] %>% mutate("cell_barcode" = rownames(.))
    metadata.tmp$log_total_counts <- log(metadata.tmp$total_counts,10)
    colnames(metadata.tmp)[1] <- 'orig.ident'
    cell_type <- ser.tmp@meta.data$celltype_L3[1]
    
  # gene selection: expressed in >50% donors, expressed in > 5% of cells
  # expression per donor -> use matrix operation
    donorMatrix = reshape2::dcast(metadata.tmp[,c('orig.ident', 'cell_barcode')], formula = orig.ident~cell_barcode)
    donorMatrix[,2:ncol(donorMatrix)] = ifelse(is.na(donorMatrix[,2:ncol(donorMatrix)]), 0, 1 )
    geMatrix = GetAssayData(ser.tmp, slot = 'counts') %>% t()
    geMatrix = (geMatrix>0)
    if(!all(rownames(geMatrix) %in% colnames(donorMatrix))){ print("cell names do not match! Check metadata and counts slot"); break}
    if(!nrow(geMatrix)==(ncol(donorMatrix)-1)){ print("cell count does not match! Check metadata and counts slot"); break}
    
    donorMatrix = donorMatrix[,c("orig.ident", rownames(geMatrix))] %>% column_to_rownames("orig.ident") %>% as.matrix() %>% Matrix(., sparse=T)
    if(!all(rownames(geMatrix) %in% colnames(donorMatrix))){ print("Second check! cell names do not match! Check metadata and counts slot"); break}
    donorMatrix2 = donorMatrix %*% geMatrix
    selected.genes = colnames(donorMatrix2)[colSums(donorMatrix2 != 0) > (length(unique(metadata.tmp$orig.ident))/2)]
    
  # expressed in >5% of total cells
    common.genes = intersect(selected.genes,
                             colnames(geMatrix)[colSums(geMatrix) > (0.05*ncol(ser.tmp))])
    all(rownames(metadata.tmp) == colnames(GetAssayData(ser.tmp, slot='count')[common.genes,]))

    # covariates matrix
    ser.tmp <- FindVariableFeatures(ser.tmp)
    ser.tmp <- ScaleData(ser.tmp)
    ser.tmp <- RunPCA(ser.tmp, npcs=20)
    pca.matrix <-  ser.tmp@reductions$pca@cell.embeddings[,1:5]
    all(rownames(metadata.tmp) == rownames(pca.matrix))
    covariates <- cbind(metadata.tmp[rownames(pca.matrix), ], pca.matrix)
    covariates = merge(clininfo, covariates, by='orig.ident') %>% column_to_rownames("cell_barcode")
    cell.order = rownames(covariates)
    
  # Write GE df
    counts <- GetAssayData(ser.tmp, slot = "counts")
    counts <- counts[common.genes, rownames(covariates), drop = FALSE]
    counts <- t(as.matrix(counts))
    all(rownames(covariates) == rownames(counts))
    ge.df <- cbind(covariates, counts)
    if(!all(rownames(ge.df) == cell.order)){print("ge df does not match cell order! check!"); break}
    outfile <- file.path(save.dir, paste0(cell_type, ".txt"))
    data.table::fwrite(data.table::data.table(cell_barcode = rownames(ge.df), ge.df),file = outfile,sep = "\t")
}

Using cell_barcode as value column: use value.var to override.

Centering and scaling data matrix

PC_ 1 
Positive:  MALAT1, MCL1, KLF6, CD82, POU2F2, CALM2, KLF2, TAGLN2, AHNAK, YPEL5 
	   FOS, ACTG1, DUSP1, CIB1, HLA-DRB5, JUN, FOSB, CRIP1, PLAC8, JUNB 
	   PLEK, IGHM, TNFRSF1B, ZFP36, DAPP1, PPP1R15A, SUB1, IER2, NR4A2, LRRFIP1 
Negative:  TCL1A, DSP, PKHD1, C10orf67, PIP, TBXT, WFIKKN1, NRXN2, UACA, IGHV4-34 
	   IGLV3-9, ST6GALNAC3, GYPA, IQCA1, CD200, IFI27, OR51A4, HOTAIR, STAM-AS1, ZNF560 
	   CCDC110, RNASE10, LSAMP, CASP5, COL7A1, IGLV3-27, PLAC4, CCDC140, ADAM29, HIST1H2BL 
PC_ 2 
Positive:  LTB, CD24, NIBAN3, IGHM, TAGLN2, CD1C, CPNE5, FAM107B, CSGALNACT1, MARCKS 
	   BIRC3, CD27, CD82, SP140, AHNAK, ITGB1, FOXP1, JCHAIN, QRSL1, SELL 
	   CKAP4, TCF4, DUSP22, AEBP1, CRIP1, TNFRSF13B, KLK1, ACTG1, GAPT, PAWR 
Negative:  NR4A2, RGS2, SRGN, ENC1, FCRL5, ZNF331, MPP6, TXNDC5, TNFRSF1B, GRASP 
	   MAP3K8, ITGAX, CD22, CD83, FCRL3, CREM, HSPB1, CEMIP2, JUNB, SCPEP1 
	   DTX1, MAC

### vcf to Plink

In [ ]:
#VCF→PLINK 格式转换 Linux
cd /media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/input/genetics/
plink2 --vcf samplePHD_snp_qc_id_chr.vcf.gz \
       --make-bed \
       --out HIV_snp_2

In [3]:
#VCF→PLINK 格式转换
library(snpStats)

plink_path <- "/media/scPBMC2_AnalysisDisk1/huangzhuoli/CIMA_r1_saigeqtl/input/genetics/10.maf01"
gt_raw <- read.plink(plink_path)

gt_map <- gt_raw$map
gt_map

载入需要的程序包：survival


载入程序包：‘survival’


The following object is masked from ‘package:future’:

    cluster




,chromosome,snp.name,cM,position,allele.1,allele.2
,<int>,<chr>,<lgl>,<int>,<chr>,<chr>
chr1_266093,1,chr1_266093,NA,266093,G,C
chr1_266225,1,chr1_266225,NA,266225,A,C
chr1_280087,1,chr1_280087,NA,280087,G,A
chr1_280485,1,chr1_280485,NA,280485,C,CT
chr1_285386,1,chr1_285386,NA,285386,G,T
chr1_768987,1,chr1_768987,NA,768987,C,T
chr1_769828,1,chr1_769828,NA,769828,C,G
chr1_771008,1,chr1_771008,NA,771008,C,T
chr1_771265,1,chr1_771265,NA,771265,C,A


In [4]:
geno_mat <- gt_raw$genotypes
geno_mat

,chr1_266093,chr1_266225,chr1_280087,chr1_280485,chr1_285386,chr1_768987,chr1_769828,chr1_771008,chr1_771265,chr1_778597,⋯,chr22_50794884,chr22_50794919,chr22_50795238,chr22_50795915,chr22_50797531,chr22_50797551,chr22_50798635,chr22_50802392,chr22_50803843,chr22_50807702
CIMA_H056,01,01,03,03,03,01,03,03,03,03,⋯,01,01,03,03,03,03,01,03,02,02
CIMA_H053,01,01,02,02,02,01,03,03,03,03,⋯,02,02,02,02,03,02,02,02,03,03
CIMA_H055,02,02,02,02,02,01,03,03,03,03,⋯,01,01,03,03,03,03,01,03,01,01
CIMA_H001,01,01,03,03,03,01,03,03,03,03,⋯,03,03,02,02,03,02,02,02,02,02
CIMA_H057,01,01,02,02,02,02,02,02,02,02,⋯,02,02,02,02,03,02,02,02,03,03
CIMA_H059,01,01,03,03,03,02,02,02,02,02,⋯,02,02,03,03,02,03,02,03,03,03
CIMA_H061,01,01,03,03,03,02,02,02,02,02,⋯,02,02,03,03,02,03,02,03,02,02
CIMA_H054,02,02,03,03,03,02,02,02,02,02,⋯,02,02,03,03,02,03,02,03,02,02
CIMA_H060,01,01,03,03,03,01,03,03,03,03,⋯,03,03,03,03,01,03,03,03,03,03
CIMA_H052,01,01,03,03,03,01,03,03,03,03,⋯,01,01,03,03,03,03,01,03,03,03


In [12]:
geno_df <- t(read_parquet("/media/scPBMC1_AnalysisDisk1/huangzhuoli/CIMA_r1/Data/413_sample_genotype.parquet"))
geno_df

E-B21100458292,2,2,0,0,0,2,0,0,0,0,⋯,2,2,0,0,0,0,2,0,1,1
E-B21103279967,2,2,1,1,1,2,0,0,0,0,⋯,1,1,1,1,0,1,1,1,0,0
E-B21105477143,1,1,1,1,1,2,0,0,0,0,⋯,2,2,0,0,0,0,2,0,2,2
E-B21106356138,2,2,0,0,0,2,0,0,0,0,⋯,0,0,1,1,0,1,1,1,1,1
E-B21106792844,2,2,1,1,1,1,1,1,1,1,⋯,1,1,1,1,0,1,1,1,0,0
E-B21107799305,2,2,0,0,0,1,1,1,1,1,⋯,1,1,0,0,1,0,1,0,0,0
E-B21108122365,2,2,0,0,0,1,1,1,1,1,⋯,1,1,0,0,1,0,1,0,1,1
E-B21116522625,1,1,0,0,0,1,1,1,1,1,⋯,1,1,0,0,1,0,1,0,1,1
E-B21116844489,2,2,0,0,0,2,0,0,0,0,⋯,0,0,0,0,2,0,0,0,0,0
E-B21122061361,2,2,0,0,0,2,0,0,0,0,⋯,2,2,0,0,0,0,2,0,0,0
E-B21122745065,2,2,0,0,0,2,0,0,0,0,⋯,0,0,0,0,2,0,0,1,0,0


In [16]:
#VCF→PLINK 格式转换
library(snpStats)

plink_path <- "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/input/genetics/HIV_snp"
gt_raw <- read.plink(plink_path)

gt_map <- gt_raw$map
gt_map

载入需要的程序包：survival


载入程序包：‘survival’


The following object is masked from ‘package:future’:

    cluster




,chromosome,snp.name,cM,position,allele.1,allele.2
,<int>,<chr>,<lgl>,<int>,<chr>,<chr>
1_16487_T_[C],1,1_16487_T_[C],NA,16487,C,T
1_16534_C_[T],1,1_16534_C_[T],NA,16534,T,C
1_17746_A_[G],1,1_17746_A_[G],NA,17746,G,A
1_49298_C_[T],1,1_49298_C_[T],NA,49298,T,C
1_51206_G_[A],1,1_51206_G_[A],NA,51206,A,G
1_51951_T_[C],1,1_51951_T_[C],NA,51951,C,T
1_54353_C_[A],1,1_54353_C_[A],NA,54353,A,C
1_54421_A_[G],1,1_54421_A_[G],NA,54421,G,A
1_54493_C_[T],1,1_54493_C_[T],NA,54493,T,C


In [17]:
geno_mat <- gt_raw$genotypes
geno_mat

,1_16487_T_[C],1_16534_C_[T],1_17746_A_[G],1_49298_C_[T],1_51206_G_[A],1_51951_T_[C],1_54353_C_[A],1_54421_A_[G],1_54493_C_[T],1_55299_C_[T],⋯,22_50797551_G_[A],22_50798635_T_[C],22_50799821_A_[C],22_50800612_G_[A],22_50801260_T_[C],22_50801268_T_[C],22_50806741_C_[T],22_50807539_C_[G],22_50807608_T_[C],22_50807614_T_[C]
HD-H134,03,03,02,03,03,03,02,02,03,02,⋯,01,02,03,03,03,03,03,03,03,03
HD-H135,03,02,03,03,03,03,03,03,03,02,⋯,03,02,03,03,03,02,03,03,03,03
HD-H136,03,02,02,03,03,03,03,02,03,02,⋯,02,03,03,02,03,02,03,03,03,03
HD-H137,02,03,03,02,03,03,03,02,03,02,⋯,03,02,03,03,03,03,02,03,03,03
HD-H138,03,03,03,03,03,03,03,02,03,02,⋯,03,02,03,03,03,03,03,03,03,03
HD-H140,03,02,03,03,03,03,03,02,03,02,⋯,03,02,03,03,03,03,03,03,03,03
HD-H141,03,02,02,03,03,03,03,02,03,02,⋯,03,03,03,03,02,03,03,03,03,03
HD-H143,03,03,03,03,03,03,03,03,03,03,⋯,02,03,03,03,03,03,03,03,03,03
HD-H145,03,03,03,03,02,02,03,02,03,02,⋯,03,02,03,03,03,03,02,03,03,03
HD-H146,03,03,02,03,03,03,03,02,03,02,⋯,02,03,03,03,03,03,03,03,03,03


In [7]:
geno_df <- t(read_parquet("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/QTL/Data/Genotype/142_sample_genotype.parquet"))
geno_df

variant_id,1_16487_T_[C],1_16534_C_[T],1_17746_A_[G],1_49298_C_[T],1_51206_G_[A],1_51951_T_[C],1_54353_C_[A],1_54421_A_[G],1_54493_C_[T],1_55299_C_[T],⋯,22_50797551_G_[A],22_50798635_T_[C],22_50799821_A_[C],22_50800612_G_[A],22_50801260_T_[C],22_50801268_T_[C],22_50806741_C_[T],22_50807539_C_[G],22_50807608_T_[C],22_50807614_T_[C]
HD-H134,0,0,1,0,0,0,1,1,0,1,⋯,2,1,0,0,0,0,0,0,0,0
HD-H135,0,1,0,0,0,0,0,0,0,1,⋯,0,1,0,0,0,1,0,0,0,0
HD-H136,0,1,1,0,0,0,0,1,0,1,⋯,1,0,0,1,0,1,0,0,0,0
HD-H137,1,0,0,1,0,0,0,1,0,1,⋯,0,1,0,0,0,0,1,0,0,0
HD-H138,0,0,0,0,0,0,0,1,0,1,⋯,0,1,0,0,0,0,0,0,0,0
HD-H140,0,1,0,0,0,0,0,1,0,1,⋯,0,1,0,0,0,0,0,0,0,0
HD-H141,0,1,1,0,0,0,0,1,0,1,⋯,0,0,0,0,1,0,0,0,0,0
HD-H143,0,0,0,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,0,0,0
HD-H145,0,0,0,0,1,1,0,1,0,1,⋯,0,1,0,0,0,0,1,0,0,0
HD-H146,0,0,1,0,0,0,0,1,0,1,⋯,1,0,0,0,0,0,0,0,0,0


In [10]:
colnames(geno_df) <- geno_df[1,]
geno_df <- geno_df[-1,]
mode(geno_df) <- "numeric"
geno_snp <- as(geno_df, "SnpMatrix")
geno_snp

,1_16487_T_[C],1_16534_C_[T],1_17746_A_[G],1_49298_C_[T],1_51206_G_[A],1_51951_T_[C],1_54353_C_[A],1_54421_A_[G],1_54493_C_[T],1_55299_C_[T],⋯,22_50797551_G_[A],22_50798635_T_[C],22_50799821_A_[C],22_50800612_G_[A],22_50801260_T_[C],22_50801268_T_[C],22_50806741_C_[T],22_50807539_C_[G],22_50807608_T_[C],22_50807614_T_[C]
HD-H134,01,01,02,01,01,01,02,02,01,02,⋯,03,02,01,01,01,01,01,01,01,01
HD-H135,01,02,01,01,01,01,01,01,01,02,⋯,01,02,01,01,01,02,01,01,01,01
HD-H136,01,02,02,01,01,01,01,02,01,02,⋯,02,01,01,02,01,02,01,01,01,01
HD-H137,02,01,01,02,01,01,01,02,01,02,⋯,01,02,01,01,01,01,02,01,01,01
HD-H138,01,01,01,01,01,01,01,02,01,02,⋯,01,02,01,01,01,01,01,01,01,01
HD-H140,01,02,01,01,01,01,01,02,01,02,⋯,01,02,01,01,01,01,01,01,01,01
HD-H141,01,02,02,01,01,01,01,02,01,02,⋯,01,01,01,01,02,01,01,01,01,01
HD-H143,01,01,01,01,01,01,01,01,01,01,⋯,02,01,01,01,01,01,01,01,01,01
HD-H145,01,01,01,01,02,02,01,02,01,02,⋯,01,02,01,01,01,01,02,01,01,01
HD-H146,01,01,02,01,01,01,01,02,01,02,⋯,02,01,01,01,01,01,01,01,01,01


In [11]:
gt_raw$genotypes <- geno_snp

In [18]:
fam <- fread("/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/input/genetics/HIV_snp.fam",
             header = FALSE)
fam[, V2 := tstrsplit(V2, "_", fixed = TRUE)[[1]]]
fam$V1 <- fam$V2
fam

V1,V2,V3,V4,V5,V6
<chr>,<chr>,<int>,<int>,<int>,<int>
HD-H134,HD-H134,0,0,0,-9
HD-H135,HD-H135,0,0,0,-9
HD-H136,HD-H136,0,0,0,-9
HD-H137,HD-H137,0,0,0,-9
HD-H138,HD-H138,0,0,0,-9
HD-H140,HD-H140,0,0,0,-9
HD-H141,HD-H141,0,0,0,-9
HD-H143,HD-H143,0,0,0,-9
HD-H145,HD-H145,0,0,0,-9


In [4]:
fwrite(fam, "/media/AnalysisDisk2/Yangshichen/0_HIV_RNA/SAIGEQTL/input/genetics/HIV_snp_2.fam",col.names = FALSE, sep = "\t")